In [1]:
# conda activate covid19
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.19.2', '1.2.2')

1) Load the raw csv file into a pandas dataframe

In [3]:
# https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario
csvName = 'data/conposcovidloc-2021-02-23'
rawDf = pd.read_csv(csvName + '.csv', 
                  header=0, 
                  index_col='Row_ID', 
                  parse_dates=['Accurate_Episode_Date','Case_Reported_Date','Test_Reported_Date','Specimen_Date'])
rawDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Row_ID,,,,,,,,,,,,,,,,,
1,2020-06-23,2020-06-27,2020-06-26,2020-06-25,60s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,2020-04-24,2020-05-01,2020-04-30,2020-04-28,40s,FEMALE,NO KNOWN EPI LINK,Resolved,NaN,2233,Grey Bruce Health Unit,101 17th Street East,Owen Sound,N4K 0A5,www.publichealthgreybruce.on.ca/,44.576196,-80.940980
3,2020-05-19,2020-05-23,2020-05-23,2020-05-22,90+,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
4,2020-04-21,2020-04-25,2020-04-25,2020-04-22,40s,MALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
5,2020-05-11,2020-05-15,2020-05-12,2020-05-11,80s,MALE,MISSING INFORMATION,Resolved,NaN,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796


In [4]:
rawDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295119 entries, 1 to 295119
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Accurate_Episode_Date      295119 non-null  datetime64[ns]
 1   Case_Reported_Date         295119 non-null  datetime64[ns]
 2   Test_Reported_Date         284570 non-null  datetime64[ns]
 3   Specimen_Date              293283 non-null  datetime64[ns]
 4   Age_Group                  295119 non-null  object        
 5   Client_Gender              295119 non-null  object        
 6   Case_AcquisitionInfo       295119 non-null  object        
 7   Outcome1                   295119 non-null  object        
 8   Outbreak_Related           65216 non-null   object        
 9   Reporting_PHU_ID           295119 non-null  int64         
 10  Reporting_PHU              295119 non-null  object        
 11  Reporting_PHU_Address      295119 non-null  object  

In [5]:
# https://www.geeksforgeeks.org/creating-a-sqlite-database-from-csv-with-python/

# Import required libraries 
import sqlite3 
import pandas as pd 

# Connect to SQLite database 
conn = sqlite3.connect(r'conposcovidloc.db') 

# Write the data to a sqlite table 
rawDf.to_sql(csvName, conn, if_exists='replace', index=True) 

# Close connection to SQLite database 
conn.close() 

2) Extract the Public Health Unit data into a csv file

In [6]:
phuIndexColumn = 'Reporting_PHU_ID'
phuInfoColumns = ['Reporting_PHU', 'Reporting_PHU_Address','Reporting_PHU_City','Reporting_PHU_Postal_Code','Reporting_PHU_Website','Reporting_PHU_Latitude','Reporting_PHU_Longitude']
phuAllColumns =  [phuIndexColumn] + phuInfoColumns
phuDf = rawDf[phuAllColumns].set_index(phuIndexColumn).drop_duplicates()
phuDf.to_csv('PublicHealthUnits.csv')
phuDf.head()

,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
Reporting_PHU_ID,,,,,,,
2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2233,Grey Bruce Health Unit,101 17th Street East,Owen Sound,N4K 0A5,www.publichealthgreybruce.on.ca/,44.576196,-80.940980
2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
2241,"Kingston, Frontenac and Lennox & Addington Pub...",221 Portsmouth Avenue,Kingston,K7M 1V5,www.kflaph.ca,44.227874,-76.525211
2266,Wellington-Dufferin-Guelph Public Health,160 Chancellors Way,Guelph,N1G 0E1,www.wdgpublichealth.ca,43.524881,-80.233743


In [7]:
phuDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 2253 to 2240
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reporting_PHU              34 non-null     object 
 1   Reporting_PHU_Address      34 non-null     object 
 2   Reporting_PHU_City         34 non-null     object 
 3   Reporting_PHU_Postal_Code  34 non-null     object 
 4   Reporting_PHU_Website      34 non-null     object 
 5   Reporting_PHU_Latitude     34 non-null     float64
 6   Reporting_PHU_Longitude    34 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.1+ KB


3) Drop the Public Health Unit Information columns from the raw data frame into a new caseDf dataframe.

In [8]:
caseDf = rawDf.drop(phuInfoColumns, axis = 1)
caseDf['Outbreak_Related'] = caseDf['Outbreak_Related'].fillna('No')
caseDf.head()

,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID
Row_ID,,,,,,,,,,
1,2020-06-23,2020-06-27,2020-06-26,2020-06-25,60s,FEMALE,CC,Resolved,No,2253
2,2020-04-24,2020-05-01,2020-04-30,2020-04-28,40s,FEMALE,NO KNOWN EPI LINK,Resolved,No,2233
3,2020-05-19,2020-05-23,2020-05-23,2020-05-22,90+,FEMALE,OB,Resolved,Yes,2253
4,2020-04-21,2020-04-25,2020-04-25,2020-04-22,40s,MALE,OB,Resolved,Yes,2253
5,2020-05-11,2020-05-15,2020-05-12,2020-05-11,80s,MALE,MISSING INFORMATION,Resolved,No,2236


In [9]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295119 entries, 1 to 295119
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Accurate_Episode_Date  295119 non-null  datetime64[ns]
 1   Case_Reported_Date     295119 non-null  datetime64[ns]
 2   Test_Reported_Date     284570 non-null  datetime64[ns]
 3   Specimen_Date          293283 non-null  datetime64[ns]
 4   Age_Group              295119 non-null  object        
 5   Client_Gender          295119 non-null  object        
 6   Case_AcquisitionInfo   295119 non-null  object        
 7   Outcome1               295119 non-null  object        
 8   Outbreak_Related       295119 non-null  object        
 9   Reporting_PHU_ID       295119 non-null  int64         
dtypes: datetime64[ns](4), int64(1), object(5)
memory usage: 24.8+ MB


In [10]:
print(caseDf.Age_Group.unique())
print(caseDf.Client_Gender.unique())
print(caseDf.Case_AcquisitionInfo.unique())
print(caseDf.Outcome1.unique())
print(caseDf.Outbreak_Related.unique())

['60s' '40s' '90+' '80s' '50s' '70s' '30s' '20s' '<20' 'UNKNOWN']
['FEMALE' 'MALE' 'GENDER DIVERSE' 'UNSPECIFIED']
['CC' 'NO KNOWN EPI LINK' 'OB' 'MISSING INFORMATION' 'TRAVEL'
 'UNSPECIFIED EPI LINK']
['Resolved' 'Fatal' 'Not Resolved']
['No' 'Yes']


4) Create dictionaries for every column you want to convert from an object to an integer.

In [11]:
ageGroup = {'UNKNOWN':0, '<20':1, '20s':2, '30s':3, '40s':4, '50s':5, '60s':6, '70s':7, '80s':8, '90+':9}
clientGender = {'UNSPECIFIED':0, 'FEMALE':1, 'MALE':2, 'GENDER DIVERSE':3}
caseAcquistionInfo = {'MISSING INFORMATION':0, 'CC':1, 'OB':2, 'TRAVEL':3, 'NO KNOWN EPI LINK':4, 'UNSPECIFIED EPI LINK':5}
outcome1 = {'Resolved':0, 'Not Resolved':1, 'Fatal':2}
outbreakRelated = {'No':0, 'Yes':1}

5) Insert these new columns into the caseDf using the dictionaries defined above.

In [12]:
%%time
caseDf['AgeGroupId'] = caseDf.apply(lambda row: ageGroup[row.Age_Group], axis=1)
caseDf['ClientGenderId'] = caseDf.apply(lambda row: clientGender[row.Client_Gender], axis=1)
caseDf['CaseAcquisitionInfoId'] = caseDf.apply(lambda row: caseAcquistionInfo[row.Case_AcquisitionInfo], axis=1)
caseDf['Outcome1Id'] = caseDf.apply(lambda row: outcome1[row.Outcome1], axis=1)
caseDf['OutbreakRelatedId'] = caseDf.apply(lambda row: outbreakRelated[row.Outbreak_Related], axis=1)
caseDf.head(10)

# This only uses 1 cpu ... 
#     mbp ...
#     CPU times: user 1min 12s, sys: 695 ms, total: 1min 13s
#     Wall time: 1min 13s

#     kauwitb ... 
#     CPU times: user 21.7 s, sys: 191 ms, total: 21.9 s
#     Wall time: 21.9 s



CPU times: user 1min 9s, sys: 544 ms, total: 1min 10s
Wall time: 1min 10s


,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,AgeGroupId,ClientGenderId,CaseAcquisitionInfoId,Outcome1Id,OutbreakRelatedId
Row_ID,,,,,,,,,,,,,,,
1,2020-06-23,2020-06-27,2020-06-26,2020-06-25,60s,FEMALE,CC,Resolved,No,2253,6,1,1,0,0
2,2020-04-24,2020-05-01,2020-04-30,2020-04-28,40s,FEMALE,NO KNOWN EPI LINK,Resolved,No,2233,4,1,4,0,0
3,2020-05-19,2020-05-23,2020-05-23,2020-05-22,90+,FEMALE,OB,Resolved,Yes,2253,9,1,2,0,1
4,2020-04-21,2020-04-25,2020-04-25,2020-04-22,40s,MALE,OB,Resolved,Yes,2253,4,2,2,0,1
5,2020-05-11,2020-05-15,2020-05-12,2020-05-11,80s,MALE,MISSING INFORMATION,Resolved,No,2236,8,2,0,0,0
6,2020-03-28,2020-04-03,2020-04-03,2020-03-30,50s,MALE,CC,Resolved,No,2236,5,2,1,0,0
7,2020-03-31,2020-04-25,2020-04-25,2020-04-21,70s,FEMALE,NO KNOWN EPI LINK,Resolved,No,2236,7,1,4,0,0
8,2020-06-04,2020-06-08,2020-06-08,2020-06-07,50s,FEMALE,OB,Resolved,Yes,2253,5,1,2,0,1
9,2020-03-31,2020-04-13,2020-04-13,2020-04-12,60s,MALE,CC,Resolved,No,2253,6,2,1,0,0


In [13]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295119 entries, 1 to 295119
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Accurate_Episode_Date  295119 non-null  datetime64[ns]
 1   Case_Reported_Date     295119 non-null  datetime64[ns]
 2   Test_Reported_Date     284570 non-null  datetime64[ns]
 3   Specimen_Date          293283 non-null  datetime64[ns]
 4   Age_Group              295119 non-null  object        
 5   Client_Gender          295119 non-null  object        
 6   Case_AcquisitionInfo   295119 non-null  object        
 7   Outcome1               295119 non-null  object        
 8   Outbreak_Related       295119 non-null  object        
 9   Reporting_PHU_ID       295119 non-null  int64         
 10  AgeGroupId             295119 non-null  int64         
 11  ClientGenderId         295119 non-null  int64         
 12  CaseAcquisitionInfoId  295119 non-null  int6

6) Drop the columns we no longer need.

In [14]:
caseDf.drop(['Accurate_Episode_Date', 'Test_Reported_Date', 'Specimen_Date', 'Age_Group','Client_Gender','Case_AcquisitionInfo','Outcome1','Outbreak_Related'], axis = 1, inplace=True)
caseDf.head(10)

,Case_Reported_Date,Reporting_PHU_ID,AgeGroupId,ClientGenderId,CaseAcquisitionInfoId,Outcome1Id,OutbreakRelatedId
Row_ID,,,,,,,
1,2020-06-27,2253,6,1,1,0,0
2,2020-05-01,2233,4,1,4,0,0
3,2020-05-23,2253,9,1,2,0,1
4,2020-04-25,2253,4,2,2,0,1
5,2020-05-15,2236,8,2,0,0,0
6,2020-04-03,2236,5,2,1,0,0
7,2020-04-25,2236,7,1,4,0,0
8,2020-06-08,2253,5,1,2,0,1
9,2020-04-13,2253,6,2,1,0,0


In [15]:
caseDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295119 entries, 1 to 295119
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Case_Reported_Date     295119 non-null  datetime64[ns]
 1   Reporting_PHU_ID       295119 non-null  int64         
 2   AgeGroupId             295119 non-null  int64         
 3   ClientGenderId         295119 non-null  int64         
 4   CaseAcquisitionInfoId  295119 non-null  int64         
 5   Outcome1Id             295119 non-null  int64         
 6   OutbreakRelatedId      295119 non-null  int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 18.0 MB


7) Save the caseDf into a csv file

In [16]:
caseDf.to_csv('PositiveCases.csv')